*Copyright: 
Prof. Dr. Christian Wartena, http://textmining.wp.hs-hannover.de/; Frieda Josi*

## Extraktion von Wortpaaren ergänzt durch Terme

Extraktion von Wortpaaren und Termen erfolgt aus den Beschriftungen und Referenzstellen der 397 Abbildungen aus Artikel, die aus Open Access Journals (NOA Projekt http://noa.wp.hs-hannover.de) stammen. Die Artikel haben meist mehrere Abbildungen, die aber nicht alle mit aufgenommen wurden. Es sind 397 einzelne txt-Dateien, angezeigt werden die Datensätze 150 bis 170. 

Kookkurenzen vom Wörter im gleichen Satz.
Lemmata statt Wörter.

Aus den Wortpaaren die Terme NN, VP, NNS. Zusätzlich  NN, VB, NNS, NP und NNP, die nicht Teil einer Nominalphrase sind.

In [2]:
import glob
import nltk
import codecs
import re
import treetaggerwrapper
from nltk.corpus import stopwords

swlist = stopwords.words('english')
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
filelist = glob.glob("doi-cap-con/*.txt")
nr_of_words = 0
fdist = nltk.FreqDist() #Worthäufigkeiten Anzahl
fdist2 = nltk.FreqDist() #Wortpaarhäufigkeiten Frequenz von Wortpaaren

for datei in filelist:  
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')#Texte aufteilen, Jeder Satz ist eine Liste von Tokens
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]#Daraus Wortpaare und Wörter extrahieren
    words = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        words = [lemma for (word,pos,lemma) in tags2 if pos[0]]#Jede Wortart+ Lemma
        word_pairs = [] 
        for i in range(len(tags2)-1) :#Bereich Liste von Zahlen 0 bis Länge von "tags2" -1. Das nächste letzte Wort soll immer angeschaut werden.
            t1 = tags2[i]# Wort i + i+1, 
            t2 = tags2[i+1]
            if t1.pos[0] and t2.pos[0]:
                l1 = t1.lemma #daraus werden die Lemma gebildet
                l2 = t2.lemma
                if l1 not in swlist and l2 not in swlist: #Stopwörter rausnehmen
                    word_pairs.append((l1,l2)) #häufigsten Wortpaare. Nur Häufigkeit.
                
        nr_of_words += len(words)
        fdist.update(words)
        fdist2.update(word_pairs)
        
    
print(fdist2.most_common(3))

[((')', '.'), 1746), (('@card@', ')'), 1608), (('<', 'math'), 1167)]


## Eingrenzung auf ein Muster

Wortartmuster für Kookkurrenzen (Wortartmuster für Nominalphrasen) 

In [3]:
import glob
import nltk
import codecs
import re
import treetaggerwrapper
import pprint
import math
from nltk.corpus import stopwords

swlist = stopwords.words('english')
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
filelist = glob.glob("doi-cap-con/*.txt")
nr_of_words = 0
fdist = nltk.FreqDist() #Worthäufigkeiten
fdist2 = nltk.FreqDist() #Wortpaarhäufigkeiten


npmuster = [('NN','NN'), #Nomen
            ('NP','NP'), #Eigennamen
            ('NN','NP'), 
            ('NP','NN'),
            ('JJ','NP'), # Adjektive
            ('JJ','NN')
           ]
    

for datei in filelist:  
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()

    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    words = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        words = [lemma for (word,pos,lemma) in tags2 if pos[0]]
        word_pairs = []
        for i in range(len(tags2)-1) :
            t1 = tags2[i]
            t2 = tags2[i+1]
            if (t1.pos,t2.pos) in npmuster:
                l1 = t1.lemma
                l2 = t2.lemma
                if l1 not in swlist and l2 not in swlist  and len(l1) >2 and len(l2) >2: #Englische Stopwörter
                    word_pairs.append((l1,l2))
        
        nr_of_words += len(words)
        fdist.update(words)
        fdist2.update(word_pairs)
print("Anzahl Wortvorkommen (tokens): ", nr_of_words)
pprint.pprint(fdist2.most_common(100))     


Anzahl Wortvorkommen (tokens):  235570
[(('wind', 'speed'), 69),
 (('time', 'series'), 69),
 (('standard', 'deviation'), 55),
 (('right', 'panel'), 53),
 (('radial', 'velocity'), 51),
 (('accumulation', 'rate'), 36),
 (('study', 'area'), 32),
 (('red', 'line'), 27),
 (('water', 'level'), 26),
 (('spindle', 'cell'), 26),
 (('blue', 'line'), 25),
 (('reference', 'blade'), 24),
 (('CTH', 'scheme'), 22),
 (('velocity', 'spectra'), 21),
 (('total', 'column'), 21),
 (('camera', 'motion'), 21),
 (('boundary', 'layer'), 20),
 (('mass', 'load'), 20),
 (('peritoneal', 'cavity'), 20),
 (('relative', 'humidity'), 20),
 (('ICEFLUX', 'scheme'), 20),
 (('diurnal', 'cycle'), 19),
 (('BARC', 'building'), 18),
 (('bone', 'marrow'), 18),
 (('tip', 'deflection'), 18),
 (('time', 'period'), 18),
 (('flash', 'rate'), 18),
 (('dental', 'pulp'), 18),
 (('transduction', 'efficiency'), 18),
 (('search', 'region'), 17),
 (('black', 'line'), 17),
 (('lightning', 'activity'), 17),
 (('particulate', 'matter'), 17),

## Extraktion von Nominalphrasen aus einzelne Datensätze

Nominalphrasen nach Muster für Extraktion verwenden.
Tokenisierung ändern: Häufigkeit und tf.idf Berechnung (Nominalphrase als einziges Token)

Aus den Wortpaaren die Terme NN, VP, NNS. Zusätzlich  NN, VB, NNS, NP und NNP, die nicht Teil einer Nominalphrase sind.

In [4]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper

df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
NPList = fdist2.keys()

def candidates(taglist):#Liste mit Tags, als Ausgabe als Liste
    cand = []
    skip = False
    for i in range(len(taglist)-1) :
        if skip:
            skip = False
            continue
        skip = False
        l1 = taglist[i].lemma # erstes Wort
        l2 = taglist[i+1].lemma# zweites Wort
        if len(l1) >2 and len(l2) >2:   #Die einzelnen Wörter der NP sollen eine Länge von über 2 Buchstaben haben
            if (l1,l2) in NPList:
                skip = True
                cand.append((l1,l2))
                if taglist[i].pos == 'NN' or taglist[i].pos == 'NP': #Wenn im Wortpaar ein NN vorkommt, als Term dazu
                    cand.append((l1))
            else:
                if taglist[i].pos == 'NN' or taglist[i].pos == 'VB' or taglist[i].pos == 'NNS': #Keine NPs, aber einzelne NN oder VB,....
                    cand.append(l1)
    w1 = taglist[-1]
    if w1.pos == 'NN'or w1.pos == 'VB' or w1.pos == 'NNS' or w1.pos == 'NP' or w1.pos == 'NNP':
        cand.append(w1)
    
    return cand


def substantive_zaehlen(datei):
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = []   
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text:
                nouns_in_text.append(substantiv)
                
    for n in nouns_in_text:
        df_n = df.get(n,0) #Der Wert von n falls n vorhanden ist, sonst 0
        df[n] = df_n + 1

for f in glob.glob("doi-cap-con/*.txt"):
    substantive_zaehlen(f)

Nominalphrasen für einen Datensatz extrahieren:

In [5]:
def extract_kw(datei,nr_of_docs):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True)
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        nouns.extend(nouns_from_sent)
    fdist = nltk.FreqDist(nouns)  
        
    for word in fdist:
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word]))
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf
            
    return fdist.most_common(15)
    

filelist = glob.glob("doi-cap-con/*.txt")
nr_of_docs = len(filelist)
for f in filelist[150:170]:
    keywords = extract_kw(f,nr_of_docs)
    pprint.pprint(f)
    pprint.pprint(keywords)
    pprint.pprint("------------------------------")

'doi-cap-con/10.1155:2016:1875357.txt'
[('stripe', 0.48992380429856003),
 (('stripe', 'size'), 0.36744285322392),
 (('time', 'slot'), 0.36744285322392),
 (('different', 'wavelength'), 0.36744285322392),
 (('switch', 'fabric'), 0.24496190214928001),
 ('slot', 0.24496190214928001),
 ('system', 0.22854509903364323),
 ('switch', 0.18209314884303246),
 ('time', 0.1528356193341453),
 ('datum', 0.13882151523162087),
 (('slot', 'size'), 0.12248095107464001),
 (('optimum', 'stripe'), 0.12248095107464001),
 (('electronic', 'control'), 0.12248095107464001),
 (('electrooptical', 'switch'), 0.12248095107464001),
 (('slot', 'duration'), 0.12248095107464001)]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[('four-ball', 0.43633838820340504),
 ('ball', 0.39301668941840845),
 (('trapezoidal', 'form'), 0.21816919410170252),
 (('typical', 'cam'), 0.21816919410170252),
 (('Four-ball', 'tribotester'), 0.21816919410170252),
 (('ball', 'cradle'), 0.21816919410170252),
 (('boundary', 

## Kategorien Mapping
15 beste Wortpaare und/oder Terme

Nominalphrase als String ohne Sonderzeichen: ,  ) ( '  in der Funktion "def wiki_cats(term)" 
Umwandeln in Kleinbuchstaben

In [6]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper
import math
import requests
from nltk.corpus import stopwords
import string 
import re

filelist = glob.glob("doi-cap-con/*.txt") #glob-Modul für Pfadnamen nach Muster
nr_of_docs = len(filelist) #Anzahl der Dateien
noun2cat = {} #Dictionary für Substantiv -> [Wikicats]
catStoplist = stopwords.words('categories') ####unsinnige Kategorien (z.B Numbers)
df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
NPList = fdist2.keys()

def candidates(taglist):#Liste mit Tags, als Ausgabe als Liste
    cand = []
    skip = False
    for i in range(len(taglist)-1) :
        if skip:
            skip = False
            continue
        skip = False
        l1 = taglist[i].lemma # erstes Wort
        l2 = taglist[i+1].lemma# zweites Wort
        if len(l1) >2 and len(l2) >2:   #Die einzelnen Wörter der NP sollen eine Länge von über 2 Buchstaben haben
            if (l1,l2) in NPList:
                skip = True
                cand.append((l1,l2))
                if taglist[i].pos == 'NN' or taglist[i].pos == 'NP': #Wenn im Wortpaar ein NN vorkommt, als Term dazu
                    cand.append((l1))
            else:
                if taglist[i].pos == 'NN' or taglist[i].pos == 'VB' or taglist[i].pos == 'NNS': #Keine NPs, aber einzelne NN oder VB,....
                    cand.append(l1)
    w1 = taglist[-1]
    if w1.pos == 'NN'or w1.pos == 'VB' or w1.pos == 'NNS' or w1.pos == 'NP' or w1.pos == 'NNP':
        cand.append(w1)
    
    return cand


def substantive_zaehlen(datei):
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = []   
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text:
                nouns_in_text.append(substantiv)
                
    for n in nouns_in_text:
        df_n = df.get(n,0) #Der Wert von n falls n vorhanden ist, sonst 0
        df[n] = df_n + 1
    

for f in glob.glob("doi-cap-con/*.txt"):
    substantive_zaehlen(f)
    
def extract_kw(datei,nr_of_docs):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        nouns.extend(nouns_from_sent)
    fdist = nltk.FreqDist(nouns)  
    
    for word in fdist:
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word]))
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf
        
    
    return fdist.most_common(15)

def wiki_cats(term):
    cats = []    
    term_string = str(term)  #Strings, damit die Zeichen ,')( entfernt werden können
    term_lower = term_string.lower() #Kleinbuchstaben, weil zweites Wort bei Wikipedia klein beginnen muss
    rechar = re.compile(r"[',()]") #ersetzen der Zeichen ,')(  zu einem Leerzeichen
    term_clear = rechar.sub("", term_lower)
     
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': term_clear,  ### Bereinigte Nominalphrasen
            'prop': 'categories',
            'clshow':'!hidden',
            'cllimit':'50'
        }
    ).json()    
    
    for pageid in response['query']['pages']:
        if 'categories' in response['query']['pages'][pageid]: 
            cats.extend([cat['title'] for cat in response['query']['pages'][pageid]['categories'] if cat['ns'] == 14 and not cat['title'].startswith('Category:Disambiguation') and not cat['title'].startswith('Category:Wikipedia articles incorporating') and cat['title'] not in catStoplist])
        return set(cats) 

def collect_nouns(flist):
    global noun2cat
    for f in flist:
        nouns = extract_kw(f, nr_of_docs)
        for (n,f) in nouns:   
            noun2cat[n] = wiki_cats(n)   

collect_nouns(filelist[150:170])


filelist = glob.glob("doi-cap-con/*.txt")
nr_of_docs = len(filelist)
for f in filelist[150:170]:
    keywords = extract_kw(f,nr_of_docs)   
    catsPair = set([c for (kw,f) in keywords for c in noun2cat.get(kw)]) 
    pprint.pprint(f)
    pprint.pprint(catsPair)
    pprint.pprint("------------------------------")
    


'doi-cap-con/10.1155:2016:1875357.txt'
{'Category:Concepts in metaphysics',
 'Category:Concepts in physics',
 'Category:Electrical components',
 'Category:Human–machine interaction',
 'Category:Physical quantities',
 'Category:SI base quantities',
 'Category:Switches',
 'Category:Systems'}
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
{'Category:Balls',
 'Category:Piping',
 'Category:Plumbing',
 'Category:Valves',
 'Category:Water industry'}
'------------------------------'
'doi-cap-con/10.1155:2016:1925827.txt'
{'Category:Artificial intelligence stubs',
 'Category:Dynamical systems',
 'Category:Models of computation',
 'Category:Stochastic processes',
 'Category:Systems theory',
 'Category:Variants of random walks'}
'------------------------------'
'doi-cap-con/10.1155:2016:1928465.txt'
{'Category:Curves',
 'Category:General topology',
 'Category:Metric geometry',
 'Category:Topology'}
'------------------------------'
'doi-cap-con/10.1155:2016:1962438.txt'
{'

### Ranking der Kategorien
Oberkategorien der Kategorien ermitteln. Anzahl der Überscheidungen = Anzahl Beziehungen
Ranking nach Anzahl

In [7]:
import requests

def wiki_cats(term):
    cats = []
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': term,
            'prop': 'categories',
            'clshow':'!hidden',
            'cllimit':'50'
        }
    ).json()
    
    for pageid in  response['query']['pages']:
        cats.extend([cat['title'] for cat in response['query']['pages'][pageid]['categories'] if cat['ns'] == 14])

    return cats # Es sind doppelte Kategorien vorhanden


def intersection(a,b):
    i = 0
    for e in a:
        if e in b:
            i += 1
    return i

def wiki_cats_sort_rel(f):
    keywords = extract_kw(f,nr_of_docs) 
    cats = set([c for (kw,f) in keywords for c in noun2cat.get(kw)]) 
    extended = {}
    relnumber = {}
    for cat in cats:
        family = [cat]
        family.extend(wiki_cats(cat))
        extended[cat] = family

    for cat in cats:
        relsize = 0
        for cat2 in cats:
            if cat == cat2:
                continue
            if intersection(extended[cat],extended[cat2]) > 0:
                relsize +=1
        relnumber[cat] = relsize
    return sorted(relnumber.items(),key = lambda x:x[1],reverse=True)
            
for f in filelist[150:170]:    
    pprint.pprint(f) 
    cts = wiki_cats_sort_rel(f)
    pprint.pprint(cts)
    pprint.pprint("------------------------------") 


'doi-cap-con/10.1155:2016:1875357.txt'
[('Category:Concepts in physics', 1),
 ('Category:Physical quantities', 1),
 ('Category:Switches', 1),
 ('Category:Electrical components', 1),
 ('Category:Systems', 1),
 ('Category:Concepts in metaphysics', 1),
 ('Category:Human–machine interaction', 0),
 ('Category:SI base quantities', 0)]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[('Category:Piping', 2),
 ('Category:Valves', 1),
 ('Category:Plumbing', 1),
 ('Category:Balls', 0),
 ('Category:Water industry', 0)]
'------------------------------'
'doi-cap-con/10.1155:2016:1925827.txt'
[('Category:Systems theory', 1),
 ('Category:Variants of random walks', 1),
 ('Category:Dynamical systems', 1),
 ('Category:Stochastic processes', 1),
 ('Category:Artificial intelligence stubs', 0),
 ('Category:Models of computation', 0)]
'------------------------------'
'doi-cap-con/10.1155:2016:1928465.txt'
[('Category:Topology', 2),
 ('Category:General topology', 1),
 ('Category:Metric